<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Persist, deploy, and score a PMML model to predict species of iris</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
  <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/customer-satisfaction-prediction/images/users_banner_2-03.png?raw=true" width="600" alt="Icon"> </th>
   </tr>
</table>

This notebook demonstrates how to store a sample Predictive Model Markup Language (PMML) model and score test data. You will use the Iris data set to predict the species of an iris flower. This data set contains measurements of the iris perianth flower. 

Some familiarity with Python is helpful. This notebook is compatible Watson Studio Desktop and Python 3.6.



## Learning goals

You will learn how to:

-  Set up the Watson Machine Learning (WML) Server.
-  Deploy a PMML model.
-  Score the deployed model.


## Table of Contents

1.	[Set up the environment](#setup)<br>
    1.1 [Set up the WML Server](#setup_WML)<br>
2.	[Persist, deploy, and score a PMML model in the Watson Machine Learning (WML) repository](#scoring)<br>
    2.1 [Persist a PMML model](#persist)<br>
    2.2 [Create an online deployment](#deploy)<br>
    2.3 [Score a test data record](#score)<br>
3.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform this setup task:

<a id="setup_WML"></a>
### 1.1 Set up the WML Server

**Connect to the Watson Machine Learning Server using the Python client**<br><br>

<div class="alert-block alert-info"><br>
To install the Watson Machine Learning Server, follow <a href="https://www.ibm.com/support/knowledgecenter/SS3PWM_1.0.0/wsj/wmls/wmls-install-over.html" target="_blank" rel="noopener no referrer">these documentation steps</a>. To connect to the WML server and find your authentication information (your credentials) follow the steps provided here in the <a href="https://www.ibm.com/support/knowledgecenter/SS3PWM_1.0.0/wsj/wmls/wmls-connect.html" target="_blank" rel="noopener no referrer">Documentation</a>.<br><br>
</div>

**Action**: Enter your WML Server credentials in the following cell.

In [1]:
# Enter your credentials here.
wml_credentials = {
    'url': '---',
    'username': '---',
    'password': '---',
    'instance_id': 'icp'
}

In [3]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [4]:
client = WatsonMachineLearningAPIClient(wml_credentials)

Obtain information about the stored models on your server.

In [5]:
# List information about the stored models.
client.repository.list_models()

----  ----  -------  ----
GUID  NAME  CREATED  TYPE
----  ----  -------  ----


List deployed models.

In [6]:
# # List the deployed models.
client.deployments.list()

----  ----  -----  -------  -------------
GUID  NAME  STATE  CREATED  ARTIFACT_TYPE
----  ----  -----  -------  -------------


<a id="scoring"></a>
## 2. Persist, deploy, and score a PMML model in the Watson Machine Learning (WML) repository

In this section, you will use the Python API client to create an online deployment and score the PMML model with a new data record.

For more information, please refer to the <a href="https://wml-api-pyclient-dev-v4.mybluemix.net/" target="_blank" rel="noopener no referrer">V4 Python client documentation</a>.


### 2.1 Persist a PMML model<a id="persist"></a>

Use `wget` to download the sample PMML model, `iris_chaid.xml` from the <a href="https://github.com/pmservice/wml-sample-models" target="_blank" rel="noopener no referrer">Git repository</a>.  

<div class="alert-block alert-info"><br>
You may need to install the <tt>wget</tt> package. To install the <tt>wget</tt> package, run the following command.<br><br></div>

In [ ]:
!pip install --upgrade wget

In [8]:
# Download the sample PMML model and the 'iris_chaid.xml' file from the Github repo.
import wget
import os

sample_dir = 'pmml_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)
    
filename=os.path.join(sample_dir, 'iris_chaid.xml')
if not os.path.isfile(filename):
    filename = wget.download('https://github.com/pmservice/wml-sample-models/raw/master/pmml/iris-species/model/iris_chaid.xml', out=sample_dir)

Store the downloaded file as *CHAID PMML model for Iris data* in the WML repository, and then list all the files stored in the repository. First, you need to create the model metadata. You can obtain the space UID (one of the MetaNames in the model metadata) by using the following cells.


<div class="alert-block alert-info"><br>
You can create your own deployment space by selecting <b>Deployment Spaces</b> from the Navigation Menu on the top left of this page.<br><br></div>

In [9]:
# Obtain the UId of your space
def guid_from_space_name(client, space_name):
    instance_details = client.service_instance.get_details()
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['guid'])

**Action:** Enter the name of your deployment space in the code below: `space_uid = guid_from_space_name(client, 'YOUR DEPLOYMENT SPACE')`.

In [10]:
# Enter the name of your deployment space here:
space_uid = guid_from_space_name(client, 'YOUR DEPLOYMENT SPACE')
print("Space UID = " + space_uid)

Space UID = 362a740c-c564-41c1-9ef2-8cab4019c5ab


In [11]:
# Model metadata
props_pmml = {
    client.repository.ModelMetaNames.NAME: 'CHAID PMML model for Iris data',
    client.repository.ModelMetaNames.RUNTIME_UID: 'pmml_4.2.1', 
    client.repository.ModelMetaNames.TYPE: 'pmml_4.2.1',
    client.repository.ModelMetaNames.SPACE_UID: space_uid
}

You need the model UID to create the deployment. You can extract the model UID from the saved model details and use it in the next section to create the deployment.

In [12]:
# Create the model artifact.
model_artifact = client.repository.store_model(filename, meta_props=props_pmml)
model_uid = client.repository.get_model_uid(model_artifact)
print("Model UID = " + model_uid)

Model UID = a0c3da12-fbb9-4a93-a1b3-bf341a0c5ad3


Get the saved model metadata from WML.

In [13]:
# Details about the model.
model_details = client.repository.get_details(model_uid)
from pprint import pprint
pprint(model_details)

{'entity': {'content_status': {'state': 'persisted'},
            'label_column': 'Species',
            'name': 'CHAID PMML model for Iris data',
            'runtime': {'href': '/v4/runtimes/pmml_4.2.1'},
            'schemas': {'input': [{'fields': [{'name': 'Petal.Width',
                                               'nullable': False,
                                               'type': 'double'},
                                              {'name': 'Sepal.Width',
                                               'nullable': False,
                                               'type': 'double'},
                                              {'name': 'Petal.Length',
                                               'nullable': False,
                                               'type': 'double'}],
                                   'id': 'default_schema_id',
                                   'type': 'struct'}],
                        'output': [{'fields': [{'name': '$R-Species'

You can list all stored models once again using the `list_models` method.

In [14]:
# Display a list of all the models.
client.repository.list_models()

------------------------------------  ------------------------------  ------------------------  ----------
GUID                                  NAME                            CREATED                   TYPE
a0c3da12-fbb9-4a93-a1b3-bf341a0c5ad3  CHAID PMML model for Iris data  2019-09-24T19:05:27.424Z  pmml_4.2.1
------------------------------------  ------------------------------  ------------------------  ----------


<div class="alert-block alert-info"><br>
From the list of downloaded files, you can see that model is successfully stored in the WML repository.<br><br></div>

### 2.2 Create an online deployment<a id="deploy"></a>

Now, create an online deployment, *Iris species prediction*, for the stored model, and list all the online deployments in the WML Repository.


In [15]:
# Deployment metadata.
deploy_meta = {
    client.deployments.ConfigurationMetaNames.NAME: "Iris species prediction",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [16]:
# Create an online deployment.
deployment_details = client.deployments.create(model_uid, meta_props=deploy_meta)



#######################################################################################

Synchronous deployment creation for uid: 'a0c3da12-fbb9-4a93-a1b3-bf341a0c5ad3' started

#######################################################################################


initializing


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='60e5a4b3-58b9-4aa1-942a-49b05cf9d8d1'
------------------------------------------------------------------------------------------------




In [17]:
# # List the deployments.
client.deployments.list()

------------------------------------  -----------------------  -----  ------------------------  -------------
GUID                                  NAME                     STATE  CREATED                   ARTIFACT_TYPE
60e5a4b3-58b9-4aa1-942a-49b05cf9d8d1  Iris species prediction  ready  2019-09-24T19:05:31.373Z  model
------------------------------------  -----------------------  -----  ------------------------  -------------


<div class="alert-block alert-info"><br>
From the list of deployed models, you can see that model was  successfully created.<br><br></div>

Now, you can check details of your deployments.

In [18]:
# Deployment UID.
deployment_uid = client.deployments.get_uid(deployment_details)
print('Deployment uid = {}'.format(deployment_uid))

Deployment uid = 60e5a4b3-58b9-4aa1-942a-49b05cf9d8d1


In [19]:
# Deployment details.
print(client.deployments.get_details(deployment_uid))

{'metadata': {'parent': {'href': ''}, 'guid': '60e5a4b3-58b9-4aa1-942a-49b05cf9d8d1', 'modified_at': '', 'created_at': '2019-09-24T19:05:31.373Z', 'href': '/v4/deployments/60e5a4b3-58b9-4aa1-942a-49b05cf9d8d1'}, 'entity': {'name': 'Iris species prediction', 'custom': {}, 'online': {}, 'description': '', 'status': {'state': 'ready', 'online_url': {'url': 'https://169.63.11.165:31843/v4/deployments/60e5a4b3-58b9-4aa1-942a-49b05cf9d8d1/predictions'}}, 'asset': {'href': '/v4/models/a0c3da12-fbb9-4a93-a1b3-bf341a0c5ad3?rev=8451365e-73d3-4de0-8ba2-a299df764654'}, 'auto_redeploy': False}}


### 2.3 Score a test data record<a id="score"></a>

Now, you can score the data and predict the species of iris flower given from the data.

In [20]:
# Score data and predict species of flower. Prepare scoring payload.
job_payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        'fields': ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width'],
        'values': [[5.1, 3.5, 1.4, 0.2]]
    }]
}
pprint(job_payload)

{'scoring_input_data': [{'fields': ['Sepal.Length',
                                    'Sepal.Width',
                                    'Petal.Length',
                                    'Petal.Width'],
                         'values': [[5.1, 3.5, 1.4, 0.2]]}]}


In [21]:
# Score data and predict species of flower. 
predictions = client.deployments.score(deployment_uid, job_payload)
print(predictions)

{'predictions': [{'fields': ['$R-Species', '$RC-Species', '$RP-Species', '$RP-setosa', '$RP-versicolor', '$RP-virginica', '$RI-Species'], 'values': [['setosa', 1.0, 1.0, 1.0, 0.0, 0.0, '1']]}]}


As we can see from the prediction, this is a species of Iris Setosa.

<a id="summary"></a>
## 3. Summary and next steps     

You successfully completed this notebook! You learned how to use Watson Machine Learning for PMML model deployment and scoring. 

### Resources <a id="resources"></a>

To learn more about configurations used in this notebook or more sample notebooks, tutorials, documentation, how-tos, and blog posts, check out these links:

<div class="alert alert-block alert-success">

<h4>IBM documentation</h4>
<br>
 <li> <a href="https://wml-api-pyclient-dev-v4.mybluemix.net" target="_blank" rel="noopener no referrer">watson-machine-learning</a></li> 
 <li> <a href="https://www.ibm.com/support/knowledgecenter/SS3PWM_1.0.0/wsj/wmls/overview.html" target="_blank" rel="noopener noreferrer">Watson Machine Learning Server</a></li>
 
<h4> IBM Samples</h4>
<br>
 <li> <a href="https://github.com/IBMDataScience/sample-notebooks" target="_blank" rel="noopener noreferrer">Sample notebooks</a></li>
 <li> <a href="https://github.com/pmservice/wml-sample-models" target="_blank" rel="noopener noreferrer">Sample models</a></li>
 
<h4> Others</h4>
<br>
 <li> <a href="https://www.python.org" target="_blank" rel="noopener noreferrer">Official Python website</a><br></li>
 </div>

### Citation

Dua, D. and Karra Taniskidou, E. (2017). [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml). Irvine, CA: University of California, School of Information and Computer Science.


### Authors

**Wojciech Sobala** is a Data Scientist at IBM.  <br><br>
**Jihyoung Kim**, Ph.D., is a Data Scientist at IBM who strives to make data science easy for everyone through Watson Studio.

Copyright © 2018, 2019 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>